In [382]:
import numpy as np

In [383]:
# https://arxiv.org/pdf/cs/0011047.pdf

In [384]:
A = [[0, 0, 1, 0, 1, 1, 0],
     [1, 0, 0, 1, 0, 0, 1],
     [0, 1, 1, 0, 0, 1, 0],
     [1, 0, 0, 1, 0, 0, 0],
     [0, 1, 0, 0, 0, 0, 1],
     [0, 0, 0, 1, 1, 0, 1]]
A = np.array(A)

In [385]:
sudoku = np.array([[0, 0, 0, 0, 0, 4, 0, 9, 0],
                   [8, 0, 2, 9, 7, 0, 0, 0, 0],
                   [9, 0, 1, 2, 0, 0, 3, 0, 0],
                   [0, 0, 0, 0, 4, 9, 1, 5, 7],
                   [0, 1, 3, 0, 5, 0, 9, 2, 0],
                   [5, 7, 9, 1, 2, 0, 0, 0, 0],
                   [0, 0, 7, 0, 0, 2, 6, 0, 3],
                   [0, 0, 0, 0, 3, 8, 2, 0, 5],
                   [0, 2, 0, 5, 0, 0, 0, 0, 0]])
# sudoku = np.array([[7, 3, 5, 6, 1, 4, 8, 9, 2],
#                    [8, 4, 2, 9, 7, 3, 5, 6, 1],
#                    [9, 6, 1, 2, 8, 5, 3, 7, 4],
#                    [2, 8, 6, 3, 4, 9, 1, 5, 7],
#                    [4, 1, 3, 8, 5, 7, 9, 2, 6],
#                    [5, 7, 9, 1, 2, 6, 4, 3, 8],
#                    [1, 5, 7, 4, 9, 2, 6, 8, 3],
#                    [6, 9, 4, 7, 3, 8, 2, 1, 5],
#                    [3, 2, 8, 5, 6, 1, 7, 4, 9]])

In [386]:
# Data object, x.
class X:
    def __init__(self, column=None, value=None):
        self.up = self.down = self.right = self.left = self
        self.column = column # Points to the column object at the head of the relevant column.
        self.value = value
    def __str__(self):
        return f'{self.column.name}:{self.value}'

# Column object, y.
class Y(X):
    def __init__(self, value=None, name=''):
        self.size = 0 # The number of 1s in the column.
        self.name = name # Symbolic identifier for printing the answers.
        
        X.__init__(self, self, value)

In [387]:
rows, cols = A.shape

root = Y(name='root')

# Use a human-readable label.
labels = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

# Prepare the header list.
curr = root
for col in range(cols):
    curr.right = Y(name=labels[col])
    curr.right.left = curr
    curr = curr.right
curr.right = root
curr.right.left = curr

header = {}
curr = root.right
while curr != root:
    header[curr.name] = curr
    curr = curr.right

In [388]:
for row in A:
    prev = None
    left = None
    for j, col in enumerate(row):
        head = header[labels[j]]
        curr = head.up
        if col == 1:
            head.size += 1
        curr.down = X(column=head, value=col)
        curr.down.up = head.up
        
        curr = curr.down
        curr.down = head
        curr.down.up = curr
        
        if prev is None:
            prev = curr
            left = curr
        else:
            prev.right = curr
            prev.right.left = prev
        prev = curr
    prev.right = left
    prev.right.left = left

In [389]:
def pprint():
    keys = header.keys()
    for key in keys:
        node = header[key]
        print(node.size)
        curr = node.down
        while curr != node:
            print(curr)
            curr = curr.down

In [390]:
pprint()

2
A:0
A:1
A:0
A:1
A:0
A:0
2
B:0
B:0
B:1
B:0
B:1
B:0
2
C:1
C:0
C:1
C:0
C:0
C:0
3
D:0
D:1
D:0
D:1
D:0
D:1
2
E:1
E:0
E:0
E:0
E:0
E:1
2
F:1
F:0
F:1
F:0
F:0
F:0
3
G:0
G:1
G:0
G:0
G:1
G:1


In [391]:
# Choose a column object.
def column_object_heuristic(root):
    '''
    Attempt to select the column object with the smallest size.
    '''
    curr = root.right
    size = float('inf')
    c = None
    while curr != root:
        if curr.size < size:
            size = curr.size
            c = curr
        curr = curr.right
    return c

In [392]:
# column_object_heuristic(root).name

In [393]:
def cover(col):
    col.right.left = col.left
    col.left.right = col.right
    i = col.down
    while i != col:
        j = i.right
        while j != i:
            j.down.up = j.up
            j.up.down = j.down
            j.column.size -= 1    
            j = j.right
        i = i.down

In [394]:
def uncover(col):
    i = col.up
    while i != col:
        j = i.left
        while j != i:
            j.column.size += 1
            j.down.up = j
            j.up.down = j
            j = j.left
        i = i.up
    col.right.left = col
    col.left.right = col

In [395]:
def search(root, k=0):
    if root == root.right:
        return
    
    col = column_object_heuristic(root)
    print(col.column.name, col.column.size)
    cover(col)
    
    r = col.down
    while r != col:
        ok = r
        print(ok.column.name)
        
        j = r.right
        while j != r:
            cover(j)
            j = j.right
        
        search(root, k+1)
        
        r = ok
        col = r.right
        
        j = r.left
        while j != r:
            uncover(j)
            j = j.left
        
        r = r.down
    uncover(col)

In [396]:
search(root)
# col = column_object_heuristic(root)
# cover(col)
# col.column.name

A 2
A


KeyboardInterrupt: 

In [ ]:
# col.right.name

In [ ]:
# node = header['B']
# curr = node.down
# while curr != node:
#     print(curr)
#     curr = node.down